In [1]:
import warnings, os, sys
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.dirname('../')))
import hn_elo as elo

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
path = '../dataset/origin/'
print(os.listdir(path))

['Data_Dictionary.xlsx', 'historical_transactions.csv', 'merchants.csv', 'new_merchant_transactions.csv', 'sample_submission.csv', 'test.csv', 'train.csv']


In [6]:
mer = pd.read_csv(path+'merchants.csv')
mer.to_hdf('../dataset/mer.h5', 'mer')

In [2]:
path = '../dataset/'
mer = pd.read_hdf(path+'mer.h5')

In [8]:
mer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 22 columns):
merchant_id                    334696 non-null object
merchant_group_id              334696 non-null int64
merchant_category_id           334696 non-null int64
subsector_id                   334696 non-null int64
numerical_1                    334696 non-null float64
numerical_2                    334696 non-null float64
category_1                     334696 non-null object
most_recent_sales_range        334696 non-null object
most_recent_purchases_range    334696 non-null object
avg_sales_lag3                 334683 non-null float64
avg_purchases_lag3             334696 non-null float64
active_months_lag3             334696 non-null int64
avg_sales_lag6                 334683 non-null float64
avg_purchases_lag6             334696 non-null float64
active_months_lag6             334696 non-null int64
avg_sales_lag12                334683 non-null float64
avg_purchases_lag12    

In [9]:
mer.isna().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [5]:
for col in mer.columns:
    dup = mer[col].duplicated().sum()
    print('''{col} : {dup}'''.format(col=col, dup=dup))

merchant_id : 63
merchant_group_id : 225305
merchant_category_id : 334372
subsector_id : 334655
numerical_1 : 333742
numerical_2 : 333749
category_1 : 334694
most_recent_sales_range : 334691
most_recent_purchases_range : 334691
avg_sales_lag3 : 331323
avg_purchases_lag3 : 234693
active_months_lag3 : 334693
avg_sales_lag6 : 330188
avg_purchases_lag6 : 199494
active_months_lag6 : 334690
avg_sales_lag12 : 329686
avg_purchases_lag12 : 161779
active_months_lag12 : 334684
category_4 : 334694
city_id : 334425
state_id : 334671
category_2 : 334690


## Obj Cols

In [12]:
objCols = []
for col in mer.columns:
    if mer[col].dtype =='O': objCols.append(col)
print(objCols)

['merchant_id', 'category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']


In [18]:
col = 'category_1'
mer[col].value_counts(dropna=False)

0    327657
1      7039
Name: category_1, dtype: int64

In [15]:
mer[col] = (mer[col]=='Y')*1
mer[col].value_counts()

0    327657
1      7039
Name: category_1, dtype: int64

In [19]:
col = 'most_recent_sales_range'
mer[col].value_counts(dropna=False)

E    177104
D    117475
C     34075
B      5037
A      1005
Name: most_recent_sales_range, dtype: int64

In [20]:
def abcde(x):
    if x =='E': return 4
    elif x=='D': return 3
    elif x=='C': return 2
    elif x=='B': return 1
    else : return 0
mer[col] = mer[col].apply(abcde)
mer[col].value_counts()

4    177104
3    117475
2     34075
1      5037
0      1005
Name: most_recent_sales_range, dtype: int64

In [21]:
col = 'most_recent_purchases_range'
mer[col].value_counts(dropna=False)

E    175309
D    119187
C     34144
B      5046
A      1010
Name: most_recent_purchases_range, dtype: int64

In [22]:
mer[col] = mer[col].apply(abcde)
mer[col].value_counts(dropna=False)

4    175309
3    119187
2     34144
1      5046
0      1010
Name: most_recent_purchases_range, dtype: int64

In [23]:
col = 'category_4'
mer[col].value_counts(dropna=False)

N    238596
Y     96100
Name: category_4, dtype: int64

In [24]:
mer[col] = (mer[col]=='Y')*1
mer[col].value_counts(dropna=False)

0    238596
1     96100
Name: category_4, dtype: int64

## Int Cols

- active month lag 3/6/12 는 연관이 있다.
- ~id 는 식별자
- num1/2 정수형으로 변환 가능할까?
    - 하고 있는데 귀찮다

In [28]:
intCols = []
for col in mer.columns:
    if mer[col].dtype=='int64': intCols.append(col)
print(set(intCols)-set(objCols))

{'merchant_category_id', 'merchant_group_id', 'state_id', 'subsector_id', 'active_months_lag12', 'active_months_lag6', 'city_id', 'active_months_lag3'}


In [44]:
col = 'numerical_1'
mer[col].describe()

count    334696.000000
mean          0.011476
std           1.098154
min          -0.057471
25%          -0.057471
50%          -0.057471
75%          -0.047556
max         183.735111
Name: numerical_1, dtype: float64

In [46]:
len(mer[col].unique())

954

In [47]:
uniqs = mer[col].unique()
uniqs = sorted(uniqs)
globalMin = [200,-1]
for i, uniq in enumerate(uniqs[1:]):
    if globalMin[0] > (uniq-uniqs[i]): 
        globalMin[0] = (uniq-uniqs[i])
        globalMin[1] = i
print(globalMin)

[0.009914899999998283, 929]


In [40]:
mer['numerical_2'].describe()

count    334696.000000
mean          0.008103
std           1.070497
min          -0.057471
25%          -0.057471
50%          -0.057471
75%          -0.047556
max         182.079322
Name: numerical_2, dtype: float64

## Float Cols

- num 1/2
- avg sales lag 3/6/12
    - na 특징 : num 1/2가 -0.057471, most ~가 둘다 4
        - 근데 num은 원래 대부분 min
        - most ~도 4가 최빈수
- avg purchases lag 3 / 6 / 12
- cat 2 : nan 때문에 float으로 구분

In [29]:
floatCols = []
for col in mer.columns:
    if mer[col].dtype == 'float64': floatCols.append(col)
print(floatCols)

['numerical_1', 'numerical_2', 'avg_sales_lag3', 'avg_purchases_lag3', 'avg_sales_lag6', 'avg_purchases_lag6', 'avg_sales_lag12', 'avg_purchases_lag12', 'category_2']


In [30]:
col = 'category_2'
mer[col].value_counts(dropna=False)

 1.0    160888
 5.0     52923
 3.0     51887
 4.0     36450
 2.0     20661
NaN      11887
Name: category_2, dtype: int64

In [49]:
11887/334695

0.03551591747710602

In [35]:
col3 = 'avg_sales_lag3'
col6 = 'avg_sales_lag6'
col12 = 'avg_sales_lag12'

id3 = mer['merchant_id'][pd.isna(mer[col3])]
id6 = mer['merchant_id'][pd.isna(mer[col6])]
id12 = mer['merchant_id'][pd.isna(mer[col12])]

print((id3 == id6).sum())
print((id6 == id12).sum())

13
13


In [38]:
mer[pd.isna(mer[col3])]

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,1,4,4,NaN,1.666667,3,NaN,4.666667,6,NaN,3.833333,12,1,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,1,4,4,NaN,0.500000,3,NaN,0.361111,6,NaN,0.347222,12,1,-1,-1,NaN
5,M_ID_a0915f62b5,20201,87,27,-0.057471,-0.057471,0,4,4,NaN,1.000000,3,NaN,3.666667,6,NaN,3.833333,12,1,160,21,5.0
6,M_ID_bfd41933db,33861,792,9,-0.057471,-0.057471,0,4,4,NaN,2.833333,3,NaN,4.833333,6,NaN,6.333333,12,0,60,16,1.0
7,M_ID_d8ff08219e,16430,529,20,-0.057471,-0.057471,1,4,4,NaN,1.000000,3,NaN,1.666667,6,NaN,1.500000,11,1,-1,-1,NaN
8,M_ID_c5b389236d,37179,813,29,-0.057471,-0.057471,0,4,4,NaN,115.000000,3,NaN,189.916667,6,NaN,197.000000,7,0,248,15,1.0
9,M_ID_d2162ed113,112122,81,29,-0.057471,-0.057471,1,4,4,NaN,1.000000,2,NaN,1.000000,2,NaN,1.000000,2,1,-1,-1,NaN
10,M_ID_492cfa500c,13462,369,27,-0.057471,-0.057471,0,4,4,NaN,inf,3,NaN,inf,6,NaN,inf,12,0,-1,9,1.0
11,M_ID_73487fed26,17123,427,27,-0.057471,-0.057471,1,4,4,NaN,inf,3,NaN,inf,6,NaN,inf,12,1,-1,-1,NaN
12,M_ID_7149162139,2118,63,27,-0.057471,-0.057471,0,4,4,NaN,inf,3,NaN,inf,6,NaN,inf,12,1,88,16,1.0


## Drop Na

In [48]:
mer = mer.dropna()
mer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322802 entries, 0 to 334695
Data columns (total 22 columns):
merchant_id                    322802 non-null object
merchant_group_id              322802 non-null int64
merchant_category_id           322802 non-null int64
subsector_id                   322802 non-null int64
numerical_1                    322802 non-null float64
numerical_2                    322802 non-null float64
category_1                     322802 non-null int32
most_recent_sales_range        322802 non-null int64
most_recent_purchases_range    322802 non-null int64
avg_sales_lag3                 322802 non-null float64
avg_purchases_lag3             322802 non-null float64
active_months_lag3             322802 non-null int64
avg_sales_lag6                 322802 non-null float64
avg_purchases_lag6             322802 non-null float64
active_months_lag6             322802 non-null int64
avg_sales_lag12                322802 non-null float64
avg_purchases_lag12       

In [50]:
mer.describe()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
count,322802.000000,322802.000000,322802.000000,322802.000000,322802.000000,322802.0,322802.000000,322802.000000,322802.000000,322802.000000,322802.000000,3.228020e+05,322802.000000,322802.000000,3.228020e+05,322802.000000,322802.000000,322802.000000,322802.000000,322802.000000,322802.000000
mean,30854.114073,423.258930,25.196700,0.012077,0.008672,0.0,3.395478,3.378913,14.164400,1.593433,2.994303,2.219137e+01,1.879788,5.948535,2.587923e+01,2.065665,11.604742,0.275890,106.744670,12.334462,2.380019
std,31580.813818,253.170306,9.806383,1.115187,1.087025,0.0,0.748723,0.753334,2438.341952,109.113434,0.093475,4.017889e+03,99.430946,0.390008,5.346628e+03,89.533281,1.508098,0.446963,107.137818,5.765773,1.562660
min,1.000000,-1.000000,-1.000000,-0.057471,-0.057471,0.0,0.000000,0.000000,-82.130000,0.333495,1.000000,-8.213000e+01,0.167045,1.000000,-8.213000e+01,0.098330,1.000000,0.000000,-1.000000,1.000000,1.000000
25%,3502.000000,222.000000,19.000000,-0.057471,-0.057471,0.0,3.000000,3.000000,0.880000,0.924661,3.000000,8.500000e-01,0.903509,6.000000,8.500000e-01,0.900498,12.000000,0.000000,-1.000000,9.000000,1.000000
50%,19663.000000,374.000000,27.000000,-0.057471,-0.057471,0.0,4.000000,4.000000,1.000000,1.017316,3.000000,1.010000e+00,1.027778,6.000000,1.030000e+00,1.044982,12.000000,0.000000,69.000000,11.000000,2.000000
75%,51353.750000,683.000000,33.000000,-0.047556,-0.047556,0.0,4.000000,4.000000,1.160000,1.146872,3.000000,1.230000e+00,1.216135,6.000000,1.290000e+00,1.267857,12.000000,1.000000,189.000000,16.000000,4.000000
max,112585.000000,891.000000,41.000000,183.735111,182.079322,0.0,4.000000,4.000000,851844.640000,61851.333333,3.000000,1.513959e+06,56077.500000,6.000000,2.567408e+06,50215.555556,12.000000,1.000000,347.000000,24.000000,5.000000


In [51]:
col = 'category_1'
mer[col].value_counts()

0    322802
Name: category_1, dtype: int64

In [59]:
def outlierPrint(col):
    print('-'*10+col+'-'*10)
    print(np.percentile(mer[col], 1))
    print(np.percentile(mer[col], 0.1))
    print(np.percentile(mer[col], 0.01))
    print(np.percentile(mer[col], 0.001))
    print(np.percentile(mer[col], 0.0001))
    print(np.percentile(mer[col], 99))
    print(np.percentile(mer[col], 99.9))
    print(np.percentile(mer[col], 99.99))
    print(np.percentile(mer[col], 99.999))
    print(np.percentile(mer[col], 99.9999))
    print('-'*50)

In [60]:
for col in mer.columns:
    if 'lag' in col : outlierPrint(col)

----------avg_sales_lag3----------
0.48
0.36
0.33
0.33000000000000007
-55.85077059
10.72
117.41393000000157
2283.4520709978005
379967.21473946667
778535.3578972796
--------------------------------------------------
----------avg_purchases_lag3----------
0.56
0.4
0.344783021522
0.3372452681959
0.33381729817342
7.33333333
50.66666667
203.34643333419072
1307.3098427653817
42444.10681012335
--------------------------------------------------
----------active_months_lag3----------
3.0
1.0
1.0
1.0
1.0
3.0
3.0
3.0
3.0
3.0
--------------------------------------------------
----------avg_sales_lag6----------
0.4
0.24
0.18
0.16
-56.34465612
18.62
211.39597000000066
3325.64684999897
742243.2136840359
1333136.5322155356
--------------------------------------------------
----------avg_purchases_lag6----------
0.48863636
0.29210010712
0.19943559024299998
0.17713690550970002
0.16953994723811
12.6
88.33333333000002
328.1396333356131
2004.0391805752242
38875.00430700628
---------------------------------

In [61]:
col = 'avg_sales_lag12'
np.log(mer[col][mer[col]>0.09]).describe()

count    322791.000000
mean          0.145322
std           0.695864
min          -2.302585
25%          -0.162519
50%           0.029559
75%           0.254642
max          14.758407
Name: avg_sales_lag12, dtype: float64

In [64]:
mer = mer[mer['avg_sales_lag12']>0]
mer = mer[mer['avg_purchases_lag12']>0]
mer['logSales12'] = np.log(mer['avg_sales_lag12'])
mer['logPurchases12'] = np.log(mer['avg_purchases_lag12'])
mer.describe()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2,logSales12,logPurchases12
count,322799.000000,322799.000000,322799.000000,322799.000000,322799.000000,322799.0,322799.000000,322799.000000,322799.000000,322799.000000,322799.000000,3.227990e+05,322799.000000,322799.000000,3.227990e+05,322799.000000,322799.000000,322799.000000,322799.000000,322799.000000,322799.000000,322799.000000,322799.000000
mean,30854.363697,423.255670,25.196841,0.012078,0.008673,0.0,3.395472,3.378908,14.164789,1.592607,2.994306,2.219184e+01,1.878938,5.948547,2.587973e+01,2.064830,11.604770,0.275893,106.744847,12.334484,2.380020,0.145259,0.151257
std,31580.852834,253.169152,9.806291,1.115192,1.087030,0.0,0.748725,0.753335,2438.353277,109.112993,0.093459,4.017907e+03,99.430365,0.389948,5.346653e+03,89.532543,1.508009,0.446964,107.137780,5.765779,1.562657,0.695972,0.582977
min,1.000000,-1.000000,-1.000000,-0.057471,-0.057471,0.0,0.000000,0.000000,0.330000,0.333495,1.000000,1.600000e-01,0.167045,1.000000,9.000000e-02,0.098330,1.000000,0.000000,-1.000000,1.000000,1.000000,-2.407946,-2.319431
25%,3502.500000,222.000000,19.000000,-0.057471,-0.057471,0.0,3.000000,3.000000,0.880000,0.924658,3.000000,8.500000e-01,0.903509,6.000000,8.500000e-01,0.900498,12.000000,0.000000,-1.000000,9.000000,1.000000,-0.162519,-0.104808
50%,19663.000000,374.000000,27.000000,-0.057471,-0.057471,0.0,4.000000,4.000000,1.000000,1.017316,3.000000,1.010000e+00,1.027778,6.000000,1.030000e+00,1.044981,12.000000,0.000000,69.000000,11.000000,2.000000,0.029559,0.043999
75%,51354.500000,683.000000,33.000000,-0.047556,-0.047556,0.0,4.000000,4.000000,1.160000,1.146855,3.000000,1.230000e+00,1.216124,6.000000,1.290000e+00,1.267857,12.000000,1.000000,189.000000,16.000000,4.000000,0.254642,0.237328
max,112585.000000,891.000000,41.000000,183.735111,182.079322,0.0,4.000000,4.000000,851844.640000,61851.333333,3.000000,1.513959e+06,56077.500000,6.000000,2.567408e+06,50215.555556,12.000000,1.000000,347.000000,24.000000,5.000000,14.758407,10.824080


## Conclusion

In [65]:
mer.columns

Index(['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'numerical_1', 'numerical_2', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12',
       'category_4', 'city_id', 'state_id', 'category_2', 'logSales12',
       'logPurchases12'],
      dtype='object')

In [67]:
mer[cols].corr()

,merchant_group_id,merchant_category_id,numerical_2,most_recent_sales_range,most_recent_purchases_range,category_4,city_id,category_2,logSales12,logPurchases12
merchant_group_id,1.000000,-0.013867,-0.057179,-0.148779,-0.118207,0.121994,-0.089864,-0.000160,-0.117467,-0.101914
merchant_category_id,-0.013867,1.000000,-0.002868,-0.047966,-0.078318,-0.017471,-0.005634,0.020924,-0.005072,-0.012306
numerical_2,-0.057179,-0.002868,1.000000,-0.165611,-0.159591,0.008423,-0.010156,-0.005635,-0.000804,-0.001081
most_recent_sales_range,-0.148779,-0.047966,-0.165611,1.000000,0.728547,-0.078777,-0.003922,-0.007389,0.214990,0.189239
most_recent_purchases_range,-0.118207,-0.078318,-0.159591,0.728547,1.000000,-0.080768,-0.019375,0.032214,0.170234,0.191700
category_4,0.121994,-0.017471,0.008423,-0.078777,-0.080768,1.000000,-0.049892,0.046426,-0.004485,-0.001073
city_id,-0.089864,-0.005634,-0.010156,-0.003922,-0.019375,-0.049892,1.000000,-0.056004,0.013911,0.016079
category_2,-0.000160,0.020924,-0.005635,-0.007389,0.032214,0.046426,-0.056004,1.000000,0.003059,0.005640
logSales12,-0.117467,-0.005072,-0.000804,0.214990,0.170234,-0.004485,0.013911,0.003059,1.000000,0.875088
logPurchases12,-0.101914,-0.012306,-0.001081,0.189239,0.191700,-0.001073,0.016079,0.005640,0.875088,1.000000


In [69]:
mer[['most_recent_sales_range', 'most_recent_purchases_range']].std()

most_recent_sales_range        0.748725
most_recent_purchases_range    0.753335
dtype: float64

In [70]:
mer[['logSales12','logPurchases12']].std()

logSales12        0.695972
logPurchases12    0.582977
dtype: float64

In [73]:
cols = ['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'numerical_2', 'most_recent_purchases_range',
       'category_4', 'category_2', 'logSales12',]
mer[cols].to_hdf('../dataset/merPrep1.h5', 'merPrep1')